In [2]:
###Setting up the environment
import sys
import os
import json

# sys.path.append(r'\\houdata04\it services\data center\autosys\ncc\python\py_est_config')
# import est_config as est

from sqlalchemy import create_engine, exc
import pandas as pd
# import cx_Oracle

# est.est_info("Libraries loaded and application starting")


In [10]:
class JobStatus:
    
    def __init__(self):
        
        self.estcc_base_sql = """
        select instance, job_group, job_name, machine, owner, joid, box_joid, run_num, ntry,
        status_epoc, status_date_string, 
        last_start, last_end,
        status_text, exit_code, event_text,    
        incident, sysid, incident_state, incident_name, assignment_group, assigned_to, 
        job_type, box_name, max_box, std_out, std_err, job_order, depth, lineage, parent_joid,     
        disp_status, bootstrap, glyph, has_condition
        """
        
        self.estcc_fail_sql = self.estcc_base_sql + " from entsched.est_cur_failures " 
        self.estcc_status_sql = self.estcc_base_sql + " from entsched.est_cur_status "
        
        self.env_
        
        
        def job_status_by_job_name(req_job_name):
    
            base_sql = """
            select joid, job_name, run_machine, status, status_text, exit_code, status_time, status_timestr,
            last_start, last_end from entsched.est_jobst where job_name = '{}' 
            """

            column_mapping = {}
    
        
j = JobStatus()
j.status_sql

'\n        select instance, job_group, job_name, machine, owner, joid, box_joid, run_num, ntry,\n        status_epoc, status_date_string, \n        last_start, last_end,\n        status_text, exit_code, event_text,    \n        incident, sysid, incident_state, incident_name, assignment_group, assigned_to, \n        job_type, box_name, max_box, std_out, std_err, job_order, depth, lineage, parent_joid,     \n        disp_status, bootstrap, glyph, has_condition\n         from entsched.est_cur_status '

In [6]:
base_sql = """
select joid, job_name, run_machine, status, status_text, exit_code, status_time, status_timestr,
last_start, last_end from est_jobst where job_name = '{}' 
"""

# db_str = est._sqlalchemydbs['RPD']['primary']
# conn_str = db_str['read']
conn_str = "sqlite:///estrprd.sqlite"

engine = create_engine(conn_str, max_identifier_length=128)
conn = engine.connect()  
jobs_df = pd.read_sql(base_sql.format('PNC-EST-DATAMART-UPDATE-JOB-WID'), conn)      

conn.close()  

col_mapping = {
    'job_name': 'jobName',
    'run_machine': 'runMachine',
    'status_code': 'statusCode',
    'status_text': 'statusText',
    'exit_code': 'exitCode',
    'status_time': 'statusEpoc',
    'status_timestr': 'statusDate',
    'last_start': 'lastStart',
    'last_end': 'lastEnd'
}

# jobs_df.rename(columns=col_mapping, inplace=True)

jobs_df


,JOID,JOB_NAME,RUN_MACHINE,STATUS,STATUS_TEXT,EXIT_CODE,STATUS_TIME,STATUS_TIMESTR,LAST_START,LAST_END
0,80288,PNC-EST-DATAMART-UPDATE-JOB-WID,ushouasysw72v,4,SUCCESS,0,1603368340,2020/10/22 07:05:40,1603368183,1603368339


In [7]:
jobs_df.to_dict(orient="records")

[{'JOID': 80288,
  'JOB_NAME': 'PNC-EST-DATAMART-UPDATE-JOB-WID',
  'RUN_MACHINE': 'ushouasysw72v',
  'STATUS': 4,
  'STATUS_TEXT': 'SUCCESS',
  'EXIT_CODE': 0,
  'STATUS_TIME': 1603368340,
  'STATUS_TIMESTR': '2020/10/22 07:05:40',
  'LAST_START': 1603368183,
  'LAST_END': 1603368339}]

In [8]:
col_list = jobs_df.columns
json_col_list = []
for cur_col in col_list:
    if "_" in cur_col:
        cur_tokens = cur_col.split("_")
        final_col = cur_tokens[0].lower()
        for cur_token in cur_tokens[1:]:
            final_col += cur_token.capitalize()
    else:
        final_col = cur_col
    json_col_list.append(final_col)    

json_col_list

jobs_df.columns = json_col_list
jobs_df
        

,JOID,JOBName,RUNMachine,STATUS,STATUSText,EXITCode,STATUSTime,STATUSTimestr,LASTStart,LASTEnd
0,80288,PNC-EST-DATAMART-UPDATE-JOB-WID,ushouasysw72v,4,SUCCESS,0,1603368340,2020/10/22 07:05:40,1603368183,1603368339


In [3]:
db_str = est._sqlalchemydbs[est.autoserv]['primary']

est.est_info(f"Extracting job list from live Autosys database {db_str['name']}...")
             
read_sql = "select * from entsched.est_bi_job_tree"
del_sql = "truncate table est_bi_job_tree"
insert_sql = "insert into est_bi_job_tree values (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16)"
             

est.est_i
             nfo(f"Query: {read_sql}") 
             
try:

             
             
except Exception as e:             
    est.est_error("There was an error extracting live info")
    est.est_error(f"Error message: {e}")
    est.est_error("*** Process has failed ***")
    est.est_exit(20)
                            
est.est_info(f"{jobs_df.shape[0]} jobs found!")   
                 
est.est_info("Attempting to load to historical database..")   
                

          
del_sql = "truncate table est_bi_job_tree"           
             
                     
try:
    engine = create_engine(db_str['hist'])
    conn = engine.connect() 
    est.est_info("    Deleting prior data....")            
    conn.execute(del_sql)      
    est.est_info("    Adding today's list to history.....")            
    jobs_df.to_sql('EST_BI_JOB_TREE', conn, if_exists='append')      
             
    conn.close()         
             
except Exception as e:             
    est.est_error("There was an error loading historical info")
    est.est_error(f"Error message: {e}")
    est.est_error("*** Process has failed ***")
    est.est_exit(20)            
             
est.est_info("All done")             
             
             


[2020-10-15 14:01:19] INFO: Extracting job list from live Autosys database ESTRDEV...
[2020-10-15 14:01:19] INFO: Query: select * from entsched.est_bi_job_tree


C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\dialects\oracle\base.py:1316: SAWarning: Oracle compatibility version (19, 0, 0, 0, 0) is known to have a maximum identifier length of 128, rather than the historical default of 30. SQLAlchemy 1.4 will use 128 for this database; please set max_identifier_length=128 in create_engine() in order to test the application with this new length, or set to 30 in order to assure that 30 continues to be used.  In particular, pay close attention to the behavior of database migrations as dynamically generated names may change. See the section 'Max Identifier Lengths' in the SQLAlchemy Oracle dialect documentation for background.
  % ((self.server_version_info,))
C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\dialects\oracle\base.py:1273: SAWarning: Could not determine compatibility version: (cx_Oracle.DatabaseError) ORA-00942: table or view does not exist
[SQL: SELECT value FROM v$parameter WHERE name = 'compatible']
(Background on this e

[2020-10-15 14:01:21] ERROR: There was an error extracting live info
[2020-10-15 14:01:21] ERROR: Error message: (cx_Oracle.DatabaseError) ORA-00942: table or view does not exist
[SQL: select * from entsched.est_bi_job_tree]
(Background on this error at: http://sqlalche.me/e/4xp6)
[2020-10-15 14:01:21] ERROR: *** Process has failed ***
[2020-10-15 14:01:21] EXIT CODE: 20


SystemExit: 20

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
help(stale_df.to_sql)